AA tree

AVL tree

Red-black tree

Scapegoat tree

Splay tree

Treap

Extent tree


---

~~B tree~~

~~B+ tree~~

B* tree

~~2-3 tree~~

Order Statistic tree


In [571]:
import bisect as b

class BTreeNode:
    def __init__(self, parent = None):
        self.vals = []
        self.children = None    
        self.parent = parent

class BTree:
    def __init__(self, knuthOrder):
        self.k = knuthOrder
        self.m = int((self.k+1)/2)
        self.root = BTreeNode()
        self.size = 0
        self.height = 1
    
    def contains(self, key):
        if self.size == 0:
            return False
        else:
            node = self.root
            while node:
                index = b.bisect_left(node.vals, key)
                if node.vals[index] == key:
                    return True
                elif node.children is not None:
                    node = node.children[index]
                else:
                    return False

    def _getLeaf(self, key):
        if self.size == 0:
            return (self.root, 0)
        else:
            node = self.root
            while node:
                index = b.bisect_left(node.vals, key)
                
                if index == len(node.vals):
                    if node.children is not None:
                        node = node.children[index]
                    else:
                        # At root
                        return (node, index)
                elif node.vals[index] == key:
                    raise Exception("Unsupported duplicate key: " + str(key))
                elif node.children is not None:
                    node = node.children[index]
                else:
                    return (node, index)

    def insert(self, val):
        node, index = self._getLeaf(val)
        self.size += 1
        if index == len(node.vals):
            node.vals.append( val)
        else:
            node.vals.insert(index, val)
        while len(node.vals) > self.k:
            # Need to split the current node
            middleVal = node.vals.pop(self.m)
            leftVals = node.vals[:self.m]
            rightVals = node.vals[self.m:]
            leftChildren, rightChildren = None,None
            
            leftNode = BTreeNode(parent = node.parent)
            leftNode.vals = leftVals
            
            node.vals = rightVals

            if node.children is not None:
                # We have exactly self.k*2 children
                leftChildren = node.children[:self.m + 1]
                for child in leftChildren:
                    child.parent = leftNode
                rightChildren = node.children[self.m + 1:]
                for child in rightChildren:
                    child.parent = node
            
            
            node.children = rightChildren
            leftNode.children = leftChildren
            
            if node.parent is None:
                # Splitting at the root
                self.height += 1
                self.root = BTreeNode()
                self.root.vals = [middleVal]
                self.root.children = [leftNode, node]
                node.parent = self.root
                leftNode.parent = self.root
                
            else:
                # Set up parent's children appropriately
                node = node.parent
                val = middleVal
                index = b.bisect_left(node.vals, val)
                node.vals.insert(index, val)
                node.children.insert(index, leftNode)
            

In [566]:
class BPlusTreeNode(BTreeNode):
    def __init__(self, parent = None, left=None, right=None):
        BTreeNode.__init__(self, parent)
        self.left = left
        self.right = right
        self.index = 0
    
class BPlusTreeNodeIterator:
    def __init__(self, node):
        self.currNode = node
        self.index = 0
    
    def __next__(self):
        if len(self.currNode.vals) == self.index:
            self.currNode = self.currNode.right
            if self.currNode is None:
                raise StopIteration
            self.index = 0
        
        val = self.currNode.vals[self.index]
        self.index += 1
        return val
            
        
class BPlusTree(BTree):
    def __init__(self, knuthOrder):
        BTree.__init__(self, knuthOrder)
        self.root = BPlusTreeNode()
        self.child = self.root
        
    def __iter__(self):
        return BPlusTreeNodeIterator(self.child)

    def insert(self, val):
        node, index = self._getLeaf(val)
        self.size += 1
        if index == len(node.vals):
            node.vals.append( val)
        else:
            node.vals.insert(index, val)
        while len(node.vals) > self.k:
            # Need to split the current node
            
            leftVals = node.vals[:self.m]
            rightVals = node.vals[self.m:]
            middleVal = rightVals[0]
                
            leftChildren, rightChildren = None,None
            leftNode = BPlusTreeNode(parent = node.parent)
            
            if node.children is not None:
                # We have exactly self.k*2 children
                # We are at an internal node
                leftChildren = node.children[:self.m + 1]
                for child in leftChildren:
                    child.parent = leftNode
                rightChildren = node.children[self.m + 1:]
                for child in rightChildren:
                    child.parent = node
                rightVals.pop(0)
            else:
                # Splitting a leaf, keep chain connected 
                leftNode.right = node
                leftNode.left = node.left
                if node.left is not None:
                    node.left.right = leftNode
                node.left = leftNode
                
            node.vals = rightVals
            node.children = rightChildren
            
            if node == self.child:
                self.child = leftNode 
            node.left = leftNode
            leftNode.vals = leftVals
            leftNode.children = leftChildren
            
            leftChildren, rightChildren = None,None

            
            if node.parent is None:
                # Splitting at the root
                self.height += 1
                self.root = BPlusTreeNode()
                self.root.vals = [middleVal]
                self.root.children = [leftNode, node]
                
                if node.children is None:
                    leftNode.right = node
                    node.left = leftNode
                
                node.parent = self.root
                leftNode.parent = self.root
                
            else:
                # Set up parent's children appropriately
                node = node.parent
                orgVal = val
                val = middleVal
                index = b.bisect_left(node.vals, val)

                    
                node.vals.insert(index, val)  
                node.children.insert(index, leftNode)
            

In [570]:
for i in range(100):
    vals = np.random.permutation([i for i in range(1000)])
    bt = BPlusTree(4)
    for v in vals:
        bt.insert(v)
    if [v for v in bt] != [i for i in range(1000)]:
        raise Exception(vals)